In [1]:
import sys
sys.path.insert(0, r'C:\Users\Public\WPy64-39100\notebooks\Anjana Shaji')
import pensionpro_api as pp
from dateutil.relativedelta import relativedelta
from datetime import datetime,timedelta,date
import numpy as np
import time
import pandas as pd
import os
from public_vault import Username, Password, OAuth
from bs4 import BeautifulSoup
from pathlib import Path

import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.base import MIMEBase
from email import encoders

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as ec
from webdriver_manager.chrome import ChromeDriverManager

from webdriver_manager.chrome import ChromeDriverManager

from O365 import Account, FileSystemTokenBackend


In [2]:
my_username = Username('DGEM')
my_pw = Password('DGEM')
today_date = date.today()
df_file_path = r"Y:\Automation\Team Scripts\Anjana Shaji\8955 SSA DF\df_filing.xlsx"
user = os.getlogin()
download_path = f'C:/Users/{user}/Downloads'
filing_path = r'Y:\Automation\Team Scripts\Anjana Shaji\8955 SSA DF\filing_report.xlsx'
email_box = 'form5500@nova401k.com'
o365_directory = Path(r'C:\Users\Public\WPy64-39100\notebooks\scheduler')
df_bad_path = r"Y:\Automation\Team Scripts\Anjana Shaji\8955 SSA DF\df_bad_status.xlsx"
cols = ['TPA Plan ID','Filed']
df_filing = pd.DataFrame(columns=cols)
df_filing.to_excel(filing_path, index=False)

In [3]:
def mail_filing_status_report(file_path):
    html_head = """
    <html xmlns:v="urn:schemas-microsoft-com:vml" xmlns:o="urn:schemas-microsoft-com:office:office" xmlns:w="urn:schemas-microsoft-com:office:word" xmlns:m="http://schemas.microsoft.com/office/2004/12/omml" xmlns="http://www.w3.org/TR/REC-html40">
    <head><META HTTP-EQUIV="Content-Type" CONTENT="text/html; charset=us-ascii"><meta name=Generator content="Microsoft Word 15 (filtered medium)">
    </head>"""
    
    toaddr = ['jworms@nova401k.com', 'msvehla@nova401k.com']
    cc_list = []

    html = html_head

    fromaddr = 'automation@nova401k.com'
    password = 'Rub73595'

    toaddrs = toaddr + cc_list

    if len(toaddrs) == 0:
        x = "No emails to send"
    else:
        msg = MIMEMultipart('alternative')

        msg['From'] = fromaddr
        msg['To'] = ','.join(toaddr)
        msg['CC'] = ','.join(cc_list)
        subject = f'Daily 8955-SSA Bad Filing Status Report'
        msg['Subject'] = subject
        part = MIMEText(html, 'html')
        msg.attach(part)
        
        part1 = MIMEBase('application', "octet-stream")
        part1.set_payload(open(file_path, "rb").read())
        encoders.encode_base64(part1)
        part1.add_header('Content-Disposition', 'attachment; filename= "8955 SSA Bad Filing Status Report.xlsx"')
        msg.attach(part1)


        with smtplib.SMTP('smtp.office365.com', 587) as server:
            server.starttls()
            server.login(fromaddr, password)
            x = server.sendmail(fromaddr, toaddrs, msg.as_string())



In [4]:
def get_latest_file(directory):
    ## Reads in the find results. This will let us use the names as selenium lookups

    fr_files = [os.path.join(directory, file) for file in os.listdir(directory) if os.path.isfile(os.path.join(directory, file))]

    # Sort the files based on modification time (latest first)
    sorted_fr_files = sorted(fr_files, key=lambda x: os.path.getmtime(x), reverse=True)

    # Get the latest modified file
    latest_find_file = sorted_fr_files[0] if sorted_fr_files else ""

    # Convert the latest_file path to string
    return str(latest_find_file)

In [5]:
plan_name_list = []
df = pp.get_worktray('Automation', get_all=True)
filt1 = df['task_name'] == '8955-SSA Filing'
filt2 = df['proj_name'] == 'Form 8955-SSA (Automated)'
df = df[filt1 & filt2]
if df.empty:
    raise SystemExit("Script is shutting down")
plan_name_list = df['plan_name'].tolist()
plan_name_list
    

APIRateLimit: The maximum allowed number of requests has been exceeded.
Retry #1... (retries=3, delay=10)
APIRateLimit: The maximum allowed number of requests has been exceeded.
Retry #2... (retries=3, delay=20)
APIRateLimit: The maximum allowed number of requests has been exceeded.
Retry #3... (retries=3, delay=30)
APIRateLimit: The maximum allowed number of requests has been exceeded.
Retry #1... (retries=3, delay=10)
APIRateLimit: The maximum allowed number of requests has been exceeded.
Retry #2... (retries=3, delay=20)


KeyboardInterrupt: 

In [6]:
len(plan_name_list)

4

In [3]:
now = datetime.now()
previous_year = str(int(now.strftime("%Y"))-1)

generated_import_file = 'AA'

# logs in using the Automation login
if generated_import_file != "No import":

    browser = webdriver.Chrome(ChromeDriverManager().install())
    # Login-----------------------------------------------------------------------
    browser.get('https://dgem.asc-net.com/ascidoc/login.aspx')

    tpakey = browser.find_element_by_name('tbTPA')
    tpakey.send_keys('asc438')

    username = browser.find_element_by_name('tbUsername')
    username.send_keys(my_username)               

    WebDriverWait(browser,10).until(ec.visibility_of_element_located((By.NAME,'tbPassword')))
    password = browser.find_element_by_name('tbPassword')
    password.send_keys(my_pw)

    loginbutton = browser.find_element_by_name('btnLogin')
    loginbutton.click()
    
    
    try:
        # Click to send verification code via "Email"
        WebDriverWait(browser,10).until(ec.visibility_of_element_located((By.XPATH,'//*[@id="MFAUC_rbEmail"]')))
        browser.find_element_by_xpath('//*[@id="MFAUC_rbEmail"]').click()

        # Click SEND
        browser.find_element_by_xpath('//*[@id="MFAUC_btnSend"]').click()
        
        if generated_import_file != "No import":
            from O365 import Account, FileSystemTokenBackend

            credentials = OAuth('o365_client_id'), OAuth('o365_secret_value')

            token_backend = FileSystemTokenBackend(token_path=o365_directory,
                                                   token_filename='oauth_token.txt')
            account = Account(credentials,
                              token_backend=token_backend,
                              scopes = ['basic','message_all','mailbox'])

            if not account.is_authenticated:
                account.authenticate()

            time.sleep(60)

            mailbox = account.mailbox(resource='automation@nova401k.com')

            inbox = mailbox.inbox_folder().get_messages()

            for message in inbox:
                msg_sender = str(message.sender)
                if (msg_sender == 'ASC (no-reply@pension-plan-emails.com)'
                    and (msg_subject := message.subject) == 'Verification Code'):
                    msg_body = message.body
                    regex = re.search(r'(?<=>)(?P<code>\d{6})(?=<)',msg_body) # find 6-digit num if between < and >
                    verification_code = regex.group('code')
                    message.mark_as_read()
                    break

            # Enter Verification code
            browser.find_element_by_xpath('//*[@id="MFAUC_tbCode"]').send_keys(verification_code)

            # Click verify
            browser.find_element_by_xpath('//*[@id="MFAUC_btnVerify"]').click()

            # Click Yes
            WebDriverWait(browser,10).until(ec.visibility_of_element_located((By.XPATH,'//*[@id="MFAUC_rbYes"]')))
            browser.find_element_by_xpath('//*[@id="MFAUC_rbYes"]').click()

            # Click "Continue"
            browser.find_element_by_xpath('//*[@id="MFAUC_btnContinue"]').click()

        else:
            exit()
    except:
        # no MFA required
        pass

In [8]:
# Download Find results: you will need to update this to select the 8955s
time.sleep(10)
errorlist = []

browser.get('https://dgem.asc-net.com/ascidoc/Find.aspx')

time.sleep(10)
form_type = browser.find_element(by='id',value='ddDocumentType').send_keys("8955-SSA")
status = browser.find_element(by='id',value='dd_SSA_status').send_keys("All")
# checkbox = browser.find_element(by='name',value='cb_SSA_statusChangedAfter').click()

time.sleep(5)
checkbox = browser.find_element(by='id',value='lbtnNext_5500').click()

time.sleep(20)

dropdown_menu = browser.find_element(by='name',value='lbActionSSA').send_keys("Export Find Results")

find_next = browser.find_element(by='id',value='lbtnClientNextSSA').click()

time.sleep(10)

link_check = False

while link_check is False:
    if "Export is complete." in browser.page_source:
        time.sleep(1)
        browser.find_element(by='link text',value='here').click()
        link_check = True
        
time.sleep(20)


In [9]:
## Reads in the find results. This will let us use the names as selenium lookups
latest_find_result = get_latest_file(download_path)
results = pd.read_html(latest_find_result, converters={"EIN":str,"Plan #":str})
df_results = results[0]

df_results['Lookup'] = df_results['Plan #'] + df_results['EIN']
os.remove(latest_find_result)
df_results

,Name,EIN,Year,Plan #,End Date,Status,Date Modified,5500 extension Date,Identifier,Lookup
0,1800 RADIATOR 401(K) PLAN,061780522,2023,002,12/31/2023,Valid,7/17/2024,NaN,12077.0,002061780522
1,2-C EQUIPMENT 401(K) PLAN,200414493,2023,001,12/31/2023,Valid,7/19/2024,NaN,4165.0,001200414493
2,"2014 FAIRVIEW AVE., LLC 401(K) PROFIT SHARING ...",260313093,2023,001,12/31/2023,Draft,7/17/2024,NaN,18201.0,001260313093
3,"2020 EXHIBITS, INC. 401(K) PROFIT SHARING PLAN",760664110,2023,001,12/31/2023,Valid,7/17/2024,NaN,5642.0,001760664110
4,"21ST CENTURY HEALTHCARE, INC. 401(K) PLAN",860733416,2023,001,12/31/2023,Valid,7/16/2024,NaN,15355.0,001860733416
...,...,...,...,...,...,...,...,...,...,...
2641,ZINSMEYER STRUCTURAL STEEL 401(K) PLAN,822782974,2023,001,12/31/2023,Valid,7/16/2024,NaN,4702.0,001822782974
2642,ZION INNOVATION CORP 401(K) PLAN,464643637,2023,001,12/31/2023,Good,7/19/2024,NaN,11184.0,001464643637
2643,"ZIVETZ, SCHWARTZ & SALTSMAN CPAS, APC 401(K) P...",952808993,2023,001,12/31/2023,Valid,7/19/2024,NaN,6090.0,001952808993
2644,Zonder Family Law 401(k) Profit Sharing Plan,824034066,2023,001,12/31/2023,Valid,7/16/2024,NaN,16476.0,001824034066


In [10]:
df_results['Identifier'] = df_results['Identifier'].fillna(0)
df_results['Identifier'] = df_results['Identifier'].astype(int)
df_results

,Name,EIN,Year,Plan #,End Date,Status,Date Modified,5500 extension Date,Identifier,Lookup
0,1800 RADIATOR 401(K) PLAN,061780522,2023,002,12/31/2023,Valid,7/17/2024,NaN,12077,002061780522
1,2-C EQUIPMENT 401(K) PLAN,200414493,2023,001,12/31/2023,Valid,7/19/2024,NaN,4165,001200414493
2,"2014 FAIRVIEW AVE., LLC 401(K) PROFIT SHARING ...",260313093,2023,001,12/31/2023,Draft,7/17/2024,NaN,18201,001260313093
3,"2020 EXHIBITS, INC. 401(K) PROFIT SHARING PLAN",760664110,2023,001,12/31/2023,Valid,7/17/2024,NaN,5642,001760664110
4,"21ST CENTURY HEALTHCARE, INC. 401(K) PLAN",860733416,2023,001,12/31/2023,Valid,7/16/2024,NaN,15355,001860733416
...,...,...,...,...,...,...,...,...,...,...
2641,ZINSMEYER STRUCTURAL STEEL 401(K) PLAN,822782974,2023,001,12/31/2023,Valid,7/16/2024,NaN,4702,001822782974
2642,ZION INNOVATION CORP 401(K) PLAN,464643637,2023,001,12/31/2023,Good,7/19/2024,NaN,11184,001464643637
2643,"ZIVETZ, SCHWARTZ & SALTSMAN CPAS, APC 401(K) P...",952808993,2023,001,12/31/2023,Valid,7/19/2024,NaN,6090,001952808993
2644,Zonder Family Law 401(k) Profit Sharing Plan,824034066,2023,001,12/31/2023,Valid,7/16/2024,NaN,16476,001824034066


In [11]:
df_submitted = df_results[df_results['Status'] == "Submitted"]
df_valid = df_results[df_results['Status'] == "Valid"]

In [12]:
df_submitted

,Name,EIN,Year,Plan #,End Date,Status,Date Modified,5500 extension Date,Identifier,Lookup
717,"DOYLE, RESTREPO, HARVIN & ROBBINS, L.L.P. 401K...",760314242,2023,001,12/31/2023,Submitted,7/18/2024,NaN,90296,001760314242
979,"GOODWILL INDUSTRIES OF DALLAS, INC. 403(B) TAX...",750800649,2023,004,12/31/2023,Submitted,7/18/2024,NaN,15973,004750800649
1071,HEALTHCARE HIGHWAYS 401(K) PLAN,272072460,2023,001,12/31/2023,Submitted,7/18/2024,NaN,3604,001272072460
1294,KARYA PROPERTY MANAGEMENT LLC 401(K) PROFIT SH...,471944709,2023,001,12/31/2023,Submitted,7/18/2024,NaN,6005,001471944709
1445,M2G VENTURES 401(K) PLAN,462390264,2023,002,12/31/2023,Submitted,7/18/2024,NaN,16708,002462390264
1546,MISSION ADVANCEMENT PROFESSIONALS 401(K) PLAN,274434340,2023,001,12/31/2023,Submitted,7/18/2024,NaN,5490,001274434340
1753,"PAXTON ACCESS, INC. RETIREMENT PLAN",412228492,2023,001,12/31/2023,Submitted,7/18/2024,NaN,5619,001412228492
1856,PROVOST UMPHREY LAW FIRM LLP 401(K) PROFIT SHA...,741820254,2023,003,12/31/2023,Submitted,7/18/2024,NaN,10237,003741820254
2440,TRK ENTERPRISES 401(K) PLAN,273555821,2023,001,12/31/2023,Submitted,7/18/2024,NaN,5732,001273555821


In [13]:
df_valid['Identifier'] = df_valid['Identifier'].astype(int)
plan_name_valid_list = []
for index, row in df.iterrows():
    plan_id = row['planid']
    plan_id = int(plan_id)
    if plan_id in df_valid['Identifier'].values:
        plan_name = df_valid.loc[df_valid['Identifier'] == plan_id, 'Name'].iloc[0]
        plan_name_valid_list.append(plan_name)
plan_name_valid_list

C:\Users\ashaji\AppData\Local\Temp\9\ipykernel_33852\3131055986.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_valid['Identifier'] = df_valid['Identifier'].astype(int)


['PROVOST UMPHREY LAW FIRM LLP 401(K) PROFIT SHARING PLAN B',
 'PALOMA ENERGY CONSULTANTS, LP 401(K) PROFIT SHARING PLAN',
 'Administrative Fiduciary Services, Inc. 401(k) Plan',
 'CHERRY PAINTING COMPANY 401(K) PROFIT SHARING PLAN']

In [14]:
len(plan_name_valid_list)

4

In [15]:
df_submitted['Identifier'] = df_submitted['Identifier'].astype(int)
plan_id_submitted_list = []
for index, row in df.iterrows():
    plan_id = row['planid']
    plan_id = int(plan_id)
    if plan_id in df_submitted['Identifier'].values:
        plan_id_submitted_list.append(plan_id)
plan_id_submitted_list

C:\Users\ashaji\AppData\Local\Temp\9\ipykernel_33852\2924507106.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_submitted['Identifier'] = df_submitted['Identifier'].astype(int)


[]

In [16]:
today_date_str = today_date.strftime("%m/%d/%Y, %H:%M:%S")
for plan_id in plan_id_submitted_list:
    projects = pp.get_projects_by_planid(plan_id,filters=f"Name eq 'Form 8955-SSA (Automated)'", expand="TaskGroups.Tasks")
    project = [project for project in projects if project['CompletedOn'] is None]
    if len(project) > 0:
        project = project[0]
    
    project_id = project['Id']
    
    for taskgroup in project['TaskGroups']:
        for task in taskgroup['Tasks']:
            if task['TaskName'] == '8955-SSA Filing' and task['DateCompleted'] is None:
                task_item = pp.get_taskitems_by_taskid(task['Id'], filters = "ShortName eq '8955-SSA File Date'")[0]
                task_item['Value'] = today_date_str
                pp.update_taskitem(task_item)
                print('updated taskitem for plan: ', plan_id)
                pp.override_task(task['Id'])

In [17]:
time.sleep(10)
errorlist = []

browser.get('https://dgem.asc-net.com/ascidoc/Find.aspx')

time.sleep(10)
form_type = browser.find_element(by='id',value='ddDocumentType').send_keys("8955-SSA")
# checkbox = browser.find_element(by='name',value='cb_SSA_statusChangedAfter').click()

time.sleep(5)
checkbox = browser.find_element(by='id',value='lbtnNext_5500').click()

time.sleep(20)

for plan in plan_name_valid_list:
    try:
        browser.find_element(by='xpath',value='//td[contains(text(),"{}")]/ancestor::tr[1]//input[@type = "checkbox"]'.format(plan)).click()
        print(plan)

    except:
        errorlist.append(plan)
        continue

dropdown_menu = browser.find_element(by='name',value='lbActionSSA').send_keys("Transmit FIRE Filing")

find_next = browser.find_element(by='id',value='lbtnClientNextSSA').click()

time.sleep(20)

PROVOST UMPHREY LAW FIRM LLP 401(K) PROFIT SHARING PLAN B
PALOMA ENERGY CONSULTANTS, LP 401(K) PROFIT SHARING PLAN
CHERRY PAINTING COMPANY 401(K) PROFIT SHARING PLAN


In [18]:
errorlist

['Administrative Fiduciary Services, Inc. 401(k) Plan']

In [19]:
time.sleep(300)
errorlist = []

browser.get('https://dgem.asc-net.com/ascidoc/Find.aspx')

time.sleep(10)
form_type = browser.find_element(by='id',value='ddDocumentType').send_keys("8955-SSA")
status = browser.find_element(by='id',value='dd_SSA_status').send_keys("Submitted (Not yet processed to IRS)")
# checkbox = browser.find_element(by='name',value='cb_SSA_statusChangedAfter').click()

time.sleep(5)
checkbox = browser.find_element(by='id',value='lbtnNext_5500').click()

time.sleep(20)

for plan in plan_name_valid_list:
    try:
        browser.find_element(by='xpath',value='//td[contains(text(),"{}")]/ancestor::tr[1]//input[@type = "checkbox"]'.format(plan)).click()
        print(plan)

    except:
        errorlist.append(plan)
        continue

time.sleep(20)

PROVOST UMPHREY LAW FIRM LLP 401(K) PROFIT SHARING PLAN B
PALOMA ENERGY CONSULTANTS, LP 401(K) PROFIT SHARING PLAN
CHERRY PAINTING COMPANY 401(K) PROFIT SHARING PLAN


In [20]:
errorlist

['Administrative Fiduciary Services, Inc. 401(k) Plan']

In [21]:
final_plan_name_list = [plan for plan in plan_name_valid_list if plan not in errorlist]
final_plan_name_list

['PROVOST UMPHREY LAW FIRM LLP 401(K) PROFIT SHARING PLAN B',
 'PALOMA ENERGY CONSULTANTS, LP 401(K) PROFIT SHARING PLAN',
 'CHERRY PAINTING COMPANY 401(K) PROFIT SHARING PLAN']

In [22]:
len(final_plan_name_list)

3

In [23]:
plan_id_list = []
for plan in final_plan_name_list:
    plan_id = df_results.loc[df_results['Name'] == plan, 'Identifier'].iloc[0]
    plan_id_list.append(plan_id)
plan_id_list

[10238, 2037, 4968]

In [25]:
today_date_str = today_date.strftime("%m/%d/%Y, %H:%M:%S")
df_filing = pd.read_excel(filing_path)
for plan_id in plan_id_list:
    projects = pp.get_projects_by_planid(plan_id,filters=f"Name eq 'Form 8955-SSA (Automated)'", expand="TaskGroups.Tasks")
    project = [project for project in projects if project['CompletedOn'] is None]
    if len(project) > 0:
        project = project[0]
    
    project_id = project['Id']
    
    for taskgroup in project['TaskGroups']:
        for task in taskgroup['Tasks']:
            if task['TaskName'] == '8955-SSA Filing' and task['DateCompleted'] is None:
                task_item = pp.get_taskitems_by_taskid(task['Id'], filters = "ShortName eq '8955-SSA File Date'")[0]
                task_item['Value'] = today_date_str
                pp.update_taskitem(task_item)
                print('updated taskitem for plan: ', plan_id)
                pp.override_task(task['Id'])
    row = {'TPA Plan ID' : plan_id,'Filed': 'Yes'}
    df_filing = pd.concat([df_filing, pd.DataFrame([row])], ignore_index=True)
print('Done!')

updated taskitem for plan:  10238
updated taskitem for plan:  2037
updated taskitem for plan:  4968
Done!


In [26]:
df_filing.to_excel(filing_path, index = False)

In [4]:
time.sleep(10)
errorlist = []

browser.get('https://dgem.asc-net.com/ascidoc/Find.aspx')
bad_flag = True
time.sleep(10)
try:
    form_type = browser.find_element(by='id',value='ddDocumentType').send_keys("8955-SSA")
    status = browser.find_element(by='id',value='dd_SSA_status').send_keys("Bad (System)")
    # checkbox = browser.find_element(by='name',value='cb_SSA_statusChangedAfter').click()

    time.sleep(5)
    checkbox = browser.find_element(by='id',value='lbtnNext_5500').click()

    time.sleep(20)

    dropdown_menu = browser.find_element(by='name',value='lbActionSSA').send_keys("Export Find Results")

    find_next = browser.find_element(by='id',value='lbtnClientNextSSA').click()

    time.sleep(10)

    link_check = False

    while link_check is False:
        if "Export is complete." in browser.page_source:
            time.sleep(1)
            browser.find_element(by='link text',value='here').click()
            link_check = True

    time.sleep(20)
except Exception as e:
    print(e)
    bad_flag = False

Message: no such element: Unable to locate element: {"method":"css selector","selector":"[name="lbActionSSA"]"}
  (Session info: chrome=126.0.6478.182)



In [6]:
## Reads in the find results. This will let us use the names as selenium lookups
if bad_flag:
    latest_find_result = get_latest_file(download_path)
    results = pd.read_html(latest_find_result, converters={"EIN":str,"Plan #":str})
    df_results = results[0]
    os.remove(latest_find_result)
    df_results
else:
    df_results = pd.DataFrame()

In [8]:
if not df_results.empty:
    print('No')
    df_results.to_excel(df_bad_path)
    mail_filing_status_report(df_bad_path)

In [33]:
browser.quit()